In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from PIL import Image
import base64
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.neighbors import NearestNeighbors

# --- Функция для кодирования изображения в Base64 для фона ---
def get_base64_of_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode()

# --- Добавление CSS для фонового изображения ---
BACKGROUND_IMAGE = 'background.jpg'  # поместите ваше фоновое изображение рядом с этим скриптом
background_base64 = get_base64_of_image(BACKGROUND_IMAGE)
st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("data:image/jpg;base64,{background_base64}");
        background-size: cover;
        background-position: center;
    }}
    </style>
    """,
    unsafe_allow_html=True
)

# --- Конфигурация путей ---
METADATA_CSV = 'images_metadata.csv'
EMBEDDINGS_NPY = 'image_embeddings.npy'

# --- Загрузка метаданных и эмбеддингов ---
@st.cache_data
def load_metadata():
    df = pd.read_csv(METADATA_CSV)
    return df

@st.cache_data
def load_embeddings():
    embs = np.load(EMBEDDINGS_NPY)
    return embs

# --- Загрузка модели backbone ---
@st.cache_resource
def load_backbone(device='cpu'):
    model = models.resnet50(pretrained=True)
    backbone = nn.Sequential(*list(model.children())[:-1])
    backbone.eval().to(device)
    return backbone

# --- Функция для получения эмбеддинга изображения ---
@st.cache_data
def get_embedding(img: Image.Image, backbone, device='cpu'):
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    tensor = transform(img.convert('RGB')).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = backbone(tensor).squeeze().cpu().numpy()
    return emb

# --- Инициализация данных ---
df = load_metadata()
embs = load_embeddings()
backbone = load_backbone(device='cpu')
nn_model = NearestNeighbors(n_neighbors=5, metric='cosine').fit(embs)

# --- Интерфейс приложения ---
st.set_page_config(page_title='Поиск похожих изображений', layout='wide')
st.title('Поиск похожих изображений в наборе PDF-картинок')

st.write('Загрузите изображение, и система выдаст топ-5 наиболее похожих из предобработанного набора.')

uploaded_img = st.file_uploader('Выберите изображение для поиска похожих', type=['png','jpg','jpeg'])
if uploaded_img:
    query_img = Image.open(uploaded_img)
    st.image(query_img, caption='Запросное изображение', width=300)

    emb_q = get_embedding(query_img, backbone)
    distances, indices = nn_model.kneighbors([emb_q])

    st.subheader('Топ-5 похожих изображений')
    cols = st.columns(5)
    for col, dist, idx in zip(cols, distances[0], indices[0]):
        info = df.iloc[idx]
        thumb = Image.open(info['image_path']).convert('RGB')
        with col:
            st.image(thumb, use_column_width=True)
            st.caption(f"{info['source_file']} | p.{info['page']} | dist={dist:.3f}")

st.header('Метаданные изображений')
st.dataframe(df[['image_path','source_file','page','chapter','width','height','is_color','class','caption']])
